# Deep Learning

We are going to apply one of the latest machine learning algorithm (Deep Learning) in prescriptive analysis. We are going to use the same data and same clinical case (AERD). The next few cells are identical to the previous exercises to read the data file and preprocess the data. Then, we need to select variables of our interests. 

In [ ]:
# The following statement will install pandas library. You may need to use pip instead of pip3
# !pip3 install pandas

## import necessary modules
import pandas as pd

aggregation = pd.read_csv(
    filepath_or_buffer= "data/aggregated.csv", # For Windows, use "\\" instead of "/"
    sep = ",", # comma separated file
    header = 0, # field names in the first line. Index starts with 0 in Python
    encoding = "utf-8", # encoding 
    #parse_dates = [1,33] # If there is any date field, put index (indices) of the field(s)
)

## Set the maximum number of columns as "None" to display all columns
pd.set_option('display.max_columns', None)

## print first 5 lines. If you need more lines, put the number of lines as an argument
## of the head function
print(aggregation.head(10)) 
print()
print(aggregation.shape) ## show the number of rows and columns

In [ ]:
aggregation[["ASPRIN53"]].describe()

In [ ]:
## 1 indicates Yes, 2 indicates No, and -1 indicates Inapplicable.

aggregation["ASPRIN53"].value_counts()

In [ ]:
aggregation["TOTEXP09"].describe()

We are going to create a positive list of attributes that are relevant to asthma and AERD. Also, please be noted that most variables are in the form of numerical variables, but they are actually categorical variables.

In [ ]:
relevant_atts = ["ADAPPT42", "ADHECR42", "AGE09X", "BMINDX53", "CHBMIX42", "FAMINC09", "ASPRIN53", "TOTEXP09" ]

## https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H129&varName=ADAPPT42
## https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H129&varName=ADHECR42
## https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H129&varName=AGE09X
## https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H129&varName=CHBMIX42
## https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H129&varName=FAMINC09
## https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H129&varName=ASPRIN53
## https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H129&varName=TOTEXP09

Before building machine learning models, we have to resolve one issue related to the body mass index. This index is separated into two variables: one for adults (BMINDX53, >17) and another for children (CHBMIX42, 6-17). Therefore, we need to combine these two variables into one variable depending on their age.

In [ ]:
aggregation[["AGE09X", "BMINDX53", "CHBMIX42"]].head(20)

In [ ]:
import numpy as np

## syntax of numpy where method: np.where(condition, value if condition is true, value if condition is false)
## combine these two variables into one variable depending on their age.
aggregation["BMINDX"] = np.where(aggregation['AGE09X'] > 17 , aggregation["BMINDX53"] , aggregation["CHBMIX42"])
aggregation["BMINDX"].head(20)

In [ ]:
## new relevant attributes
relevant_atts = ["ADAPPT42", "ADHECR42", "AGE09X", "BMINDX", "FAMINC09", "ASPRIN53", "TOTEXP09" ]

In [ ]:
mask_with_values = (aggregation["ADAPPT42"] > 0) & (aggregation["ADHECR42"] > 0) & (aggregation[ "AGE09X"] > 0) & \
                   (aggregation["BMINDX"] > 0) & (aggregation["FAMINC09"] > 0) & (aggregation["ASPRIN53"] > 0) & (aggregation["TOTEXP09"] > 0)  

## https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H129&varName=ADAPPT42

In [ ]:
filtered = aggregation[mask_with_values][relevant_atts]

# reset index for rows that were removed

filtered.reset_index(inplace = True, drop= True)
filtered.shape

### Outlier Detection

In the filtering process above, we simply removed all instances when any attribute's value is smaller or equal to zero. In the actual project, you have to apply a more sophisticated approach such as imputation. This is to use the most information from your data. In the cells below, I introduce outlier detection based on z-score. There are other methods such as IQR score, too.

In [ ]:
filtered.head(10)

In [ ]:
# get column names
filtered.columns

In [ ]:
# ASPRIN53 is a categorical variable, we cannot apply z-transformation.
# Get the column index of this variable.
filtered.columns.get_loc("ASPRIN53")

In [ ]:
import numpy as np
from scipy import stats

## We create indices that exclude categorical variable
positions = list(range(0,5))
positions.extend(list(range(6,7)))

z = np.abs(stats.zscore(filtered.iloc[:, positions]))
print(z)

The above numbers represent z scores of our data. It is difficult to say which data point is an outlier. Let’s try and define a threshold to identify an outlier. To be conservative, we will use 3 as a threshold. We can exclude 0.3% of data.

In [ ]:
threshold = 3
print(np.where(z > 3))

The first array contains the list of row numbers, and the second array corresponding column numbers that have a Z-score higher than 3, which mean they are: z[2][4], z[54][1], z[123][3], and so on.

We may want to remove tuples in which the most attributes are outliers. We can remove all tuples regardless of the number of attributes that are outliers. For this exercise, we adopt a very conservative approach. Therefore, we count the frequency of tuples that includes outliers.

In [ ]:
unique_elements, counts_elements = np.unique(np.where(z > 3)[0], return_counts=True)
print(np.asarray((unique_elements, counts_elements)))

The first list represents the row number when the z score is larger than 3. The second list represents the frequency of the corresponding row. The row 165, 176, and 751 have the largest number of outliers. Therefore, we want to remove these tuples from the dataset.

In [ ]:
filtered.index

In [ ]:
filtered = filtered.drop([ 165, 176, 751],  axis=0)

print(filtered.shape)

Now, we are almost ready for the fun analysis except for one categorical variable. We need to convert this variable(s) to dummy variables. 

In [ ]:
dummy = pd.get_dummies(filtered["ASPRIN53"])
dummy.head(10)

In [ ]:
# change column names
dummy.columns = ["ASPIRIN", "No_ASPIRIN"]
dummy.head(10)

In [ ]:
## new relevant attributes without "ASPRIN53",
relevant_atts = ["ADAPPT42", "ADHECR42", "AGE09X", "BMINDX", "FAMINC09",  "TOTEXP09" ]

filtered = pd.concat([filtered[relevant_atts], dummy], axis = 1)
filtered.head(10)

### Deep Learning Classification
Now it's time to create a deep learning model. Before that, we need to create a train and test dataset. Also, we need to separate a dependent variable from independent variables. 


In [ ]:
## syntax of numpy where method: np.where(condition, value if condition is true, value if condition is false)
## combine these two variables into one variable depending on their age.
## Sklearn takes integer as label
filtered["RISK"] = np.where(filtered['TOTEXP09'] >= filtered['TOTEXP09'].median() , 1 , 0)
filtered[['TOTEXP09', 'RISK']].head(20)

In [ ]:
from sklearn.model_selection import train_test_split

# separate a dependent variable from independent variables.

filtered = filtered.drop('TOTEXP09',  axis=1)

## independent variables
X = filtered.loc[:, filtered.columns != "RISK" ]

## dependent variable
## Specify the data type. Before specifying, the type was unknown
Y = filtered["RISK"].astype('int')

print(X.head())
print(Y.head())


## test_size will set the ratio of the test dataset
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=101)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

#### Things to consider

We are going to utilize the Keras library to implement a Deep Learning model. In Keras, a model is defined as a series of layers.

We start with a Sequential model and gradually add layers until we are satisfied with our network architecture. 

The first step is to guarantee that the input layer has the appropriate number of input features. This can be provided by setting the input dim option to 7 for the 7 input variables when establishing the first layer. 

How can we know how many layers there are and what types they are? 

This is a difficult question to answer. We can utilize heuristics, and the ideal network structure is frequently discovered by trial and error testing.  In general, you'll need a network large enough to capture the problem's structure. 

In this example, we'll use a three-layer fully connected network topology. 

The Dense class is used to define fully connected layers. The number of neurons or nodes in the layer can be specified as the first argument, and the activation function can be specified as the second argument. 

On the first two layers, we'll use the rectified linear unit (ReLU) activation function, and in the output layer, we'll utilize the Sigmoid function. 

Sigmoid and Tanh activation functions were once the preferred activation functions for all layers. These days, the ReLU activation function is used to improve performance. With a default threshold of 0.5, we utilize a sigmoid on the output layer to ensure that our network output is between 0 and 1 and easy to transfer to either a probability of class 1 or a hard classification of either class.

We can piece it all together by adding each layer: 

- The input dim=7 option specifies that the model expects rows of data with seven variables. 
- The relu activation function is used in the first hidden layer, which comprises 12 nodes. 
- The relu activation function is used in the second hidden layer, which includes 8 nodes. 
- The sigmoid activation function is used in the output layer, which contains only one node.

In [ ]:
! pip3 install keras
! pip3 install tensorflow

# first neural network with keras tutorial
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

# define the keras model
model = Sequential()
model.add(Dense(12, input_dim=7, activation='relu'))
# model.add(Dense(24, activation='relu'))
# model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


We can compile the model now that it has been defined. 

The model is compiled using efficient numerical libraries (the so-called backend) such as Theano or TensorFlow. The backend automatically selects the optimum way to represent the network for training and prediction on your hardware, whether it's a CPU, GPU, or distributed architecture. 

We must specify some more properties required for training the network when compiling. Remember that when we train a network, we're looking for the optimum collection of weights to map our dataset's inputs to outputs. 

The optimizer is used to search through different weights for the network and any optional metrics we would like to collect. We must specify the loss function to use to evaluate a set of weights, and the optimizer is used to search through different weights for the network and any optional metrics we would like to collect.

We'll utilize cross entropy as the loss argument in this situation. This loss is known as "binary crossentropy" in Keras and is used to solve binary classification issues. 

The optimizer will be "adam," an effective stochastic gradient descent technique. Because it automatically tunes itself and produces good results in a wide range of problems, this is a popular variation of gradient descent. 

Finally, because this is a classification problem, we'll collect and report classification accuracy, which will be defined by the metrics input.

In [ ]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Our model has been defined and compiled, and it is now ready for efficient computing. 

Now it's time to put the model to the test on some real-world data. 

By using the fit() function on the model, we may train or fit our model on our loaded data. 

Training takes place in epochs, with each epoch divided into batches. 

- Epoch: A single run across the training dataset's rows. 
- Batch: Before weights are adjusted, the model considers one or more samples within an epoch. 
One epoch is made up of one or more batches, depending on the batch size chosen, and the model can be used for multiple epochs.

We'll run for a short number of epochs (150) and a batch size of 10 for this problem. 

By trial and error, these configurations can be determined. We want to train the model sufficiently so that it learns a decent (or good enough) mapping of input data rows to output classification. The model will always have some inaccuracy, but for a particular model configuration, the amount of error will eventually level out. Model convergence is the term for this.

In [ ]:

# fit the keras model on the dataset
model.fit(X, Y, epochs=150, batch_size=10)

In [ ]:

# evaluate the keras model
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))





Credit: https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/